# Evaluation of a Grain Boundary Detection Method Using Cellular Automata
*Rundong Jiang, 6/26/19*


## Table of contents

1. [Learning outcomes](#Learning-outcomes)
2. [Recap: Cellular automata (CA)](#Recap:-Cellular-automata-(CA))
3. [Application: Grain boundary detection](#Application:-Grain-boundary-detection)
4. [CA setup](#CA-setup)
5. [Complete MATLAB implementation](#Complete-MATLAB-implementation)
6. [Evaluation](#Evaluation)
9. [Discussion](#Discussion)
10. [Conclusion](#Conclusion)
11. [Further learning](#Further-learning)
12. [References](#References)

## Learning outcomes 
[Back to top](#Table-of-contents)  

By the end of this lesson, you will be able to:  
* Use cellular automata to detect grain boundaries in MATLAB 
* Understand the advantages and limitations of this method  
* Enumerate potential improvements based on the specific case  

## Recap: Cellular automata (CA)
[Back to top](#Table-of-contents)  

A __cellular automaton__ is an N-dimensional grid of __cells__ with __states__ that update iteratively according to a set of __state transition rules__. There are mainly two types of __neighborhoods__ that determine which neighboring cells affect the new state of the cell: __Moore neighborhood__ (left) and __Von Neumann neighborhood__ (right). (Image credit: Wikipedia)
<table><tr>
    <td><img src="https://upload.wikimedia.org/wikipedia/en/d/d2/CA-Moore.png" alt="grain boundary" style="width: 300px;"/></td>
    <td><img src="https://upload.wikimedia.org/wikipedia/en/5/56/CA-von-Neumann.png" alt="grain boundary" style="width: 300px;"/></td>
</tr></table>


## Application: Grain boundary detection
[Back to top](#Table-of-contents)  

How is CA useful for materials scientists? While CA is commonly used for material simulation of various scales, we will focus on one particular application: grain boundary detection.  
<table><tr>
    <td><img src="img/5A_20x.jpg" alt="grain boundary" style="width: 300px;"/></td>
    <td><img src="img/5A_20x2.jpg" alt="grain boundary" style="width: 300px;"/></td>
</tr></table>
The image on a left is a typical micrograph of steel, and we can extract useful quantitative data such as grain size distribution from its microstructure. The process of grain size characterization is usually done by software packages such as ImageJ, but before we can perform any analysis, the image needs to go through a series of pre-processing to isolate the grain boundaries into a clean image, like the one on the right, that's readable by the software.
<img src="img/5_grainsize_measurement.gif" alt="grain boundary" style="width: 300px;"/>
The above gif (Credit: Aalto University Wiki) illustrates the typical image pre-processing steps involved, with one of them being manually tracing the grain boundaries, a labor-intensive process that you might have had to gone through in your other materials science classes or labs.
<img src="img/manual.jpg" alt="grain boundary" style="width: 300px;"/>
Is there a way to automate this process? Gorsevski et al. (2012) proposed a grain boundary detection method using CA.

## CA setup
[Back to top](#Table-of-contents)  

### Basic information

__Cells__: _<font color='green'>2D M\*N grid</font>_  
__Neighborhood__: _<font color='green'>Moore sq(2)</font>_  
__States__: _<font color='green'>equal to \# of intensities (0-255 for 8-bit grayscale images)</font>_  
__State Transition Rules__:
>_<font color='green'>If difference in intensity is smaller than ε for all neighbors, change to 0 -> background pixel</font>_  
>_<font color='green'>Otherwise, stay the same -> edge pixel</font>_

### Reading the image file
The first thing to do is to read the image file. An (intensity) image is represented as an M\*N array of a certain number (e.g. 256) of intensities, which can be translated to cells and states directly. MATLAB comes with some handy functions such as `imread`, `isempty` and `ind2rgb` to help us with image processing.

```octave
[A,map] = imread('INSERT NAME OF IMAGE FILE'); % read image
if ~isempty(map)
    A = ind2rgb(A,map); % convert to RGB truecolor if image is indexed
end
A = double(A); % double is easier to process than unsigned integer
```
If the code above runs successfully, our image is now stored in matrix `A` as an M\*N\*3 array (due to having 3 RGB bands).
### Initialization
We then initialize other variables that are relevant for the CA, as well as variables that are particular to the state transition rules, such as `epsilon`.
```octave
steps = 1; % total number of time steps
time = 0; % current time
placex = 0; % current x coordinate of cell
placey = 0; % current y coordinate of cell
neighs = zeros(8,1); % states of the 8 neighbors on the Moore sq(2) lattice
me = 0; % current cell state, i.e. (x,y)

epsilon =36; % threshold that controls the quality of grain boundary detection
```

Here, we can use a `for` loop to cycle through all 3 RGB bands of the image, but we will select only one band for ease of demonstration.
```octave
for k = 1:1 % choose one of the RGB bands
    oldcells = A(:,:,k); % store only one band of the image as the initial CA configuration
    newcells = oldcells; % duplicate initial CA configuration for the next time step
```
If you'd like, you can let MATLAB draw the initial time step of the CA as an image.
```octave
    imshow(uint8(oldcells),'InitialMagnification','fit') % convert back to uint8 to display the image
    title('1'); % set title to be the current time step
    drawnow;
```

### Iterating through time
Now, we are ready to iteratively update the cells using the state transition rules for each time step. Typically, another ```for``` loop will be used to control the time step. Although for this particular CA implementation, we can safely skip over this step because it only requires 1 time step, this ```for``` loop is still included in the code to conform with the typical CA setup.
```octave
    for time = 1:steps
```

### Iterating through the CA
For each time step, we cycle through all the rows and columns of the CA and change the states of each cell using two more `for` loops. Note that both loops start from 2 and end at width-1 or height-1 because we reserve the first and last rows/columns as "buffer" rows/columns so that all cells, including edge cells, have 8 neighbors. It is also customary to exclude edges of the image when doing size calculation, so the buffer section can be justified.
```octave
      
        for placey = 2:size(A,2)-1
            for placex = 2:size(A,1)-1
```
### Iterating through neighbor cells
There are 9 cells that we care about: ```me``` and my 8 neighbors on the Moore sq(2) lattice.
```octave
                me = oldcells(placex,placey);
                neighs(1) = oldcells(placex-1,placey); %left
                neighs(2) = oldcells(placex,placey+1); %above
                neighs(3) = oldcells(placex+1,placey); %right
                neighs(4) = oldcells(placex,placey-1); %below
                neighs(5) = oldcells(placex-1,placey-1); %top left
                neighs(6) = oldcells(placex-1,placey+1); %top right
                neighs(7) = oldcells(placex+1,placey-1); %bottom left
                neighs(8) = oldcells(placex+1,placey+1); %bottom right
 ```           
### Updating cell using state transition rule(s)
>For each cell, if the absolute intensity difference is lower than ```epsilon``` for all neighbors, change its state to 0.

```octave            
                if (sum((abs(neighs-me)<epsilon))==length(neighs))
                   newcells(placex,placey) = 0;
                end
          
            end % finish iterating through each column
        end % finish iterating through each row
```

Now that the timestep is done, we replace the old CA configuration with the new one.
```octave
        oldcells = newcells;
```    

### Drawing the new configuration
```octave
%       figure(2); % enable this line to draw in a new figure
        imshow(uint8(oldcells),'InitialMagnification','fit')
        title(num2str(time)); % use the current time step as title
        drawnow;
    end
end
```

## Complete MATLAB implementation
[Back to top](#Table-of-contents)  

The entire MATLAB code is reproduced below. Go ahead and run the code!

```octave
% Reading the image file
[A,map] = imread('img/test.gif'); % read image
if ~isempty(map)
    A = ind2rgb(A,map); % convert to RGB truecolor if image is indexed
end
A = double(A); % double is easier to process than unsigned integer

% Initialization
steps = 1; % total number of time steps
time = 0; % current time
placex = 0; % current x coordinate of cell
placey = 0; % current y coordinate of cell
neighs = zeros(8,1); % states of the 8 neighbors on the Moore sq(2) lattice
me = 0; % current cell state, i.e. (x,y)

epsilon =36; % threshold that controls the quality of grain boundary detection

for k = 1:1 % choose one of the RGB bands
    oldcells = A(:,:,k); % store only one band of the image as the initial CA configuration (cell states)
    newcells = oldcells; % duplicate initial CA configuration for the next time step
    
    imshow(uint8(oldcells),'InitialMagnification','fit') % convert back to uint8 to display the image
    title('1'); % set title to be the current time step
    drawnow;

%Iterating through time
    for time = 1:steps
        
%Iterating through the CA

        for placey = 2:size(A,2)-1
            for placex = 2:size(A,1)-1
                
% Iterating through neighbor cells
                me = oldcells(placex,placey);
                neighs(1) = oldcells(placex-1,placey); %left
                neighs(2) = oldcells(placex,placey+1); %above
                neighs(3) = oldcells(placex+1,placey); %right
                neighs(4) = oldcells(placex,placey-1); %below
                neighs(5) = oldcells(placex-1,placey-1); %top left
                neighs(6) = oldcells(placex-1,placey+1); %top right
                neighs(7) = oldcells(placex+1,placey-1); %bottom left
                neighs(8) = oldcells(placex+1,placey+1); %bottom right
                
% Updating cell
                if (sum((abs(neighs-me)<epsilon))==length(neighs))
                   newcells(placex,placey) = 0;
                end

            end % finish iterating through each column
        end % finish iterating through each row
        
        oldcells = newcells;
        
        figure(1);
        imshow(uint8(oldcells),'InitialMagnification','fit')
        title(num2str(time)); % use the current time step as title
        drawnow;
    end
end
```

## Evaluation
[Back to top](#Table-of-contents)  

### Case 1: Test image
Before testing with actual micrographs, let's test the code on a standard edge detection test image (below) to make sure the code works. Change the image in the `imread` function in line 1 to be `'img/test.gif'`, set `epsilon = 10`, and run the code in MATLAB.
<img src="img/test.gif" alt="grain boundary" style="width: 300px;"/>

Here are the sample CA result with `epsilon = 10` (left) and the benchmark result from SUSAN edge detector (right):
<table><tr>
    <td><img src="img/test_result.png" alt="grain boundary" style="width: 300px;"/></td>
    <td><img src="img/img75.gif" alt="grain boundary" style="width: 300px;"/></td>
</tr></table>

__Observation__: _<font color='green'>Notice that the CA result from a standard test image rivals that of other edge detectors; however, it did miss one horizontal edge and left some oblique edges disconnected.</font>_

### Case 2: High-quality micrograph
Now that the code is working properly, let's try detecting the grain boundaries on a high-quality micrograph (the color has been inverted to fit the CA rules) with our CA.
<img src="img/5_inverted.jpg" alt="grain boundary" style="width: 300px;"/>

Change the image in the `imread` function in line 1 to be `'img/5_inverted.jpg'`, set `epsilon = 12`, and run the code in MATLAB. Also try adjusting the value of `epsilon` to get the best result.  

Here are the sample CA results with `epsilon = 12` (left), `epsilon = 24` (middle) and `epsilon = 36` (right).
<table><tr>
    <td><img src="img/5_CA12.png" alt="grain boundary" style="width: 300px;"/></td>
    <td><img src="img/5_CA24.png" alt="grain boundary" style="width: 300px;"/></td>
    <td><img src="img/5_CA36.png" alt="grain boundary" style="width: 300px;"/></td>
</tr></table>

__Observation__: _<font color='green'></font>_

### Case 3: Low-quality micrograph
How well will the CA perform on this low-quality micrograph (the color has been inverted to fit the CA rules)?
<img src="img/5A_20x_inverted.jpg" alt="grain boundary" style="width: 300px;"/>

Change the image in the `imread` function in line 1 to be `'img/5A_20x_inverted.jpg'`, set `epsilon = 36`, and run the code in MATLAB. Also try adjusting the value of `epsilon` to get the best result.  

Here are the sample CA results with `epsilon = 24` (left) and `epsilon = 36` (right).
<table><tr>
    <td><img src="img/5A_CA24.png" alt="grain boundary" style="width: 300px;"/></td>
    <td><img src="img/5A_CA36.png" alt="grain boundary" style="width: 300px;"/></td>
</tr></table>

__Observation__: _<font color='green'></font>_

## Discussion
[Back to top](#Table-of-contents)  


__What are some advantages of using CA for grain boundary detection?__  
* _<font color='green'>Implementation is easy and straightforward</font>_  
* _<font color='green'>Not affected by blurriness</font>_  
* _<font color='green'>Does not require image enhancement</font>_  
* _<font color='green'></font>_  

__How about disadvantages?__  
* _<font color='green'>Naïve CA yields mediocre results that are not ready for analysis</font>_
* _<font color='green'>Cannot replace manual tracing either</font>_  
* _<font color='green'>Severely affected by noises and image quality, thus incompatible with more complex micrographs</font>_  
* _<font color='green'>Requires serious pre- and post-processing</font>_  
* _<font color='green'></font>_  


__What caveats should we bear in mind when using CA for grain boundary detection?__  
* _<font color='green'>Result is dependent on selection of ε, which is arbitrary and relies on trial and error</font>_
* _<font color='green'></font>_

__Where can we make improvements?__
* _<font color='green'>Use in combination with other models/algorithms</font>_
* _<font color='green'>Genetic algorithm for optimizing state transition rules</font>_
* _<font color='green'>Machine learning</font>_
* _<font color='green'>Use other CA implementations (e.g. seed-growing)</font>_
* _<font color='green'>Implement post-processing steps (linking, thinning) in CA and strive for fully automated grain boundary detection</font>_
* _<font color='green'></font>_





## Conclusion
[Back to top](#Table-of-contents)  

We have seen how computational methods like CA have the potential to solve interesting problems for materials scientists, such as grain boundary detection. However, we have also seen the limitations and intricacies that come with these methods. What are your biggest takeaways from this lesson?  
* _<font color='green'>CA can isolate parts of the grain boundaries, but cannot detect grain boundaries on its own</font>_
* _<font color='green'>CA requires the aid of other image processing techniques in order to achieve better results</font>_
* _<font color='green'>CA has the potential to improve by using more advanced algorithms and optimizing state transition rules</font>_

## Further learning
[Back to top](#Table-of-contents)  

There are many image pre-processing techniques that improve the detection result, including noise reduction, image enhancement, etc. Gorsevski et al. (2012) also used two techniques:

### Median filter
a median filter is used to filter out noise pixels (background noises, pores, cracks) while maintaining grain boundary structure. It is basically another CA with the same cells and states. The state transition rule is that each cell becomes the median value of its neighborhood (including itself); the wider the neighborhood, the blurrier the result.

Below is a MATLAB implementation of the median filter. `hw` is the half-window size of the median filter. E.g. if the median filter is 5\*5, then window size would be 5, and the half-window size would be 2. `hw` should be initialized along with other CA-related variables before any loops.

Can you figure out how to add this segment of code into the original code? Try to perform grain boundary detection with the added median filter and see if the results improve.
```octave
for placey = 1+hw:size(A,2)-hw
             for placex = 1+hw:size(A,1)-hw
                 window = oldcells(placex-hw:placex+hw,placey-hw:placey+hw);
                 newcells(placex,placey) = median(window(:));
             end
         end
             
         oldcells = newcells;
         imshow(uint8(oldcells),'InitialMagnification','fit') % b/w
         title(num2str(time-0.5));
         drawnow; 
```
### Thresholding  
A thresholding step highlights the grain boundaries by setting all pixels with intensity greater than a threshold to the maximum intensity (e.g. 255). The thresholding rule can either be absolute (i.e. intensity >
% certain value) ,relative (has >3 positive neighbors) or both.

Below is a MATLAB implementation of thresholding. Can you figure out how to adjust the threshold, and how to add this segment of code into the original code? Try to perform grain boundary detection with the added thresholding step and see if the results improve.
```octave
         for placey = 2:size(A,2)-1
             for placex = 2:size(A,1)-1
                 me = oldcells(placex,placey);
                 neighs(1) = oldcells(placex-1,placey); %left
                 neighs(2) = oldcells(placex,placey+1); %above
                 neighs(3) = oldcells(placex+1,placey); %right
                 neighs(4) = oldcells(placex,placey-1); %below
                 neighs(5) = oldcells(placex-1,placey-1); %top left
                 neighs(6) = oldcells(placex-1,placey+1); %top right
                 neighs(7) = oldcells(placex+1,placey-1); %bottom left
                 neighs(8) = oldcells(placex+1,placey+1); %bottom right

                 if (me>0) % OR if (sum(neighs~=0)>3)
                     newcells(placex,placey) = 255;
                 else
                     newcells(placex,placey) = 0;
                 end
             end
         end
         oldcells = newcells;
         imshow(uint8(oldcells),'InitialMagnification','fit')
         title('result');
         drawnow;  
```

## References
[Back to top](#Table-of-contents)  

Gorsevski, P. V., Onasch, C. M., Farver, J. R., & Ye, X. (2012). Detecting grain boundaries in deformed rocks using a cellular automata approach. Computers & Geosciences, 42, 136-142.  
Smith, S. M., & Brady, J. M. (1997). SUSAN—a new approach to low level image processing. International journal of computer vision, 23(1), 45-78.